In [1]:
# David Ouyang 3/2/2020

# 3/2/20
# [x] Create loss to constrain subsequent frames, IOU, and DSC

# 3/3/20
# [x] Train with different losses

# 3/5/20
# [x] Train with Adam optimizer instead of SGD

# 3/12/20
# [x] Constrained Loss training

# 3/15/20
# [x] Troubleshoot the backprop of constrained loss, remake videos

# 3/18/20
# [x] Hyperparameter search at 50 epochs, best visual performance with L2 loss and weight around 0.9
# Current run with diff instead of diff / x / y (much bigger constrainedLoss term)

import echonet
import os
import collections
import numpy as np
import tqdm
import math 
import skimage.segmentation
import scipy
import torch
import torchvision
import seaborn as sns
import matplotlib.pyplot as plt
import re
import PIL.Image
import cv2
import dataset #  https://medium.com/@grvsinghal/speed-up-your-python-code-using-multiprocessing-on-windows-and-jupyter-or-ipython-2714b49d6fac
import random
import dataset
import variousLossFunctions

SQLtracings = "C:\\Datasets\\RVWork\\SQLPull-RVMasks"
SQLvideos = "C:\\Datasets\\RVWork\\SQLPull-DownsampledFilesWithTracings"



ModuleNotFoundError: No module named 'dataset'

In [2]:
mean, std = echonet.utils.get_mean_and_std(echonet.datasets.Echo(split="train"))
print(mean,std)

#mean = np.array([28.951515,28.914696,28.896002], dtype = np.float32)
#std = np.array([47.857174,47.831146,47.798138], dtype = np.float32)

#print(mean.dtype)

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:19<00:00,  1.97s/it]


[32.14274  31.699886 31.5132  ] [49.326927 48.895126 48.61154 ]


In [9]:
# constrainedloss testing
# search just training on CE

#bestRVweights = "C:\\Datasets\RVWork\\RV_deeplab_50_SQLguide_True_AugmentTrue_best.pt"
bestLVweights = "C:\\Users\\Windows\\Dropbox\\Echo Research\\CodeBase\\EchoNetDynamic-Weights\\deeplabv3_resnet50_random.pt"

numWork = 10
numberOfSubsequentFrames = 5
bs = 45
numEpochs = 50

for weightOfConstraint in [0, 0.25, 0.1, 0.5, 0.75, 0.9, 1]: #[0.8, 0.85, 0.9, 0.95]: #  0.5, 0.1, 0, 0.9, 1 ]: #1, 0,  0.9, #   1000, 100,  10, 1]:
    for CrossEntropyLoss in [1]:
        for SonoData in [False]:
            for doAugmentation in [False]:
                for doPretrain in [False]:


                    train_dataset = dataset.Dataset(SQLtracings, SQLvideos, mean, std, split="train", fuzzyAugmentation=doAugmentation,constrainSubsequent=numberOfSubsequentFrames,useSonoData=False)

                    if SonoData:
                        sono_dataset = dataset.Dataset(SQLtracings, SQLvideos, mean, std, split="train", fuzzyAugmentation=doAugmentation,constrainSubsequent=numberOfSubsequentFrames,useSonoData=True)
                        train_dataloader = torch.utils.data.DataLoader(torch.utils.data.ConcatDataset([train_dataset,sono_dataset]), batch_size = bs, num_workers = numWork, shuffle = True, pin_memory = True, drop_last = True)
                    else:
                        train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size = bs, num_workers = numWork, shuffle = True, pin_memory = True, drop_last = True)

                    val_dataset = dataset.Dataset(SQLtracings, SQLvideos, mean, std, split="val", fuzzyAugmentation=False,constrainSubsequent=numberOfSubsequentFrames,useSonoData=False)
                    val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size = bs, num_workers = numWork, shuffle = True, pin_memory = True, drop_last = True)

                    dataloaders = {'train':train_dataloader, 'val':val_dataloader}

                    device = torch.device("cuda")

                    model = torchvision.models.segmentation.deeplabv3_resnet50(pretrained = False, aux_loss = False)
                    model.classifier[-1] = torch.nn.Conv2d(model.classifier[-1].in_channels, 1, kernel_size = model.classifier[-1].kernel_size)

                    if device.type == "cuda":
                        model = torch.nn.DataParallel(model)
                    model.to(device)

                    if doPretrain:
                        print("load from weights")
                        checkpoint = torch.load(bestLVweights)#LVWeights) #currentRVweights)#
                        model.load_state_dict(checkpoint['state_dict'])

                    #optim = torch.optim.SGD(model.parameters(), lr=1000, momentum=0.9)
                    learning_rate = 1e-2
                    optim = torch.optim.Adam(model.parameters(), lr=learning_rate)
                    #optim = torch.optim.Adagrad(model.parameters(), lr=100)

                    bestLoss = 1000000

                    with open("LossAcrossEpochs_Pretrain"+str(doPretrain)+"_Sono"+str(SonoData)+"_Augment"+str(doAugmentation)+ "_withCrossEntropyOnly"+ str(CrossEntropyLoss) + "_withL2ConstrainedLoss"+ str(weightOfConstraint) + "_"+str(numEpochs)+"_Adam.csv", 'w') as output:
                        for epoch in range(numEpochs):
                            for phase in dataloaders.keys():

                                total = 0
                                n = 0
                                dsctotal = 0
                                cetotal = 0
                                constrainedTotal = 0

                                with torch.set_grad_enabled(phase != 'val'):
                                    with tqdm.tqdm(total = len(dataloaders[phase])) as progressbar:
                                        for (i, (x,y)) in enumerate(dataloaders[phase]):

                                            x = x.to(device)
                                            y = y.to(device)
                                            #print(y.min(), y.max())
                                            #print(x.shape)

                                            #print("input shape", x.shape)

                                            # reshape to be bs x frames indepedent predictions on (3,112,112) images
                                            x = torch.transpose(x, 1, 4)
                                            #print(x.shape)
                                            x = x.reshape(bs * numberOfSubsequentFrames,  112, 112, 3).transpose(1,2)
                                            x = torch.transpose(x, 1, 3).type(torch.float)

                                            #print("new input shape", x.shape)

                                            p = model(x)["out"]

                                            #print("predictions shape", p.shape)

                                            constrainedLoss = variousLossFunctions.constrainSubsequentLoss(p, bs, numberOfSubsequentFrames)
                                            dscloss = x.shape[0] / numberOfSubsequentFrames - variousLossFunctions.diceLoss(p[0::numberOfSubsequentFrames, 0, :, :], y,)
                                            celoss = torch.nn.functional.binary_cross_entropy_with_logits(p[0::numberOfSubsequentFrames, 0, :, :], y, reduction = "sum") / 112 / 112

                                            loss =  (1-weightOfConstraint) * celoss + weightOfConstraint * constrainedLoss
                                            #CrossEntropyLoss * torch.nn.functional.binary_cross_entropy_with_logits(p[0::numberOfSubsquentFrames, 0, :, :], y, reduction = "sum") / 112 / 112 + variousLossFunctions.constrainSubsequentLoss(p, bs, numberOfSubsequentFrames).item()#x.shape[0] - variousLossFunctions.diceLoss(p[:, 0, :, :], y,) +
                                            
#                                            loss = loss.item()
                                            total += loss#.item()
                                            dsctotal += dscloss#.item()
                                            cetotal += celoss#.item()
                                            constrainedTotal += constrainedLoss

                                            n += x.shape[0] / numberOfSubsequentFrames

                                            #print(loss, total, n, flush = True)

                                            progressbar.set_postfix_str(phase + "{:.1f} total{:.4f} dsc{:.4f} ce{:.4f} constrain{:.4f}  ({:.4f})".format( epoch, total / n , dsctotal / n , cetotal / n , constrainedTotal / n, loss / x.size(0) * numberOfSubsequentFrames )) #/ 112 / 112
                                            progressbar.update()

                                            if(phase != 'val'):
                                                optim.zero_grad()
                                                loss.backward()
                                                optim.step()

                                        output.write(phase + "," + str(epoch) + "," + str(total.item() / n ) + "," + str(dsctotal.item() / n ) + "," + str(cetotal.item() / n ) + ',' + str(constrainedTotal.item() / n) + "\n") #/ 112 / 112

                                        if phase == 'val':
                                            currentRVweights = "C:\\Datasets\RVWork\\RV_deeplab_50_Pretrain"+str(doPretrain)+"_Sono"+str(SonoData)+"_Augment"+str(doAugmentation)+ "_crossEntropyOnly"+ str(CrossEntropyLoss) + "_withL2ConstrainedLoss"+ str(weightOfConstraint)  + "_"+str(numEpochs)+"_Adam_current.pt"
                                            torch.save(model.state_dict(), currentRVweights)

                                            if (total / n ) < bestLoss: #/ 112 / 112)
                                                bestLoss = (total / n ) # / 112 / 112)
                                                print("this is the best so far", str(bestLoss), flush = True)
                                                bestRVweights = "C:\\Datasets\RVWork\\RV_deeplab_50_Pretrain"+str(doPretrain)+"_Sono"+str(SonoData)+"_Augment"+str(doAugmentation)+ "_crossEntropyOnly"+ str(CrossEntropyLoss) +"_withL2ConstrainedLoss"+ str(weightOfConstraint)  + "_"+str(numEpochs)+"_Adam_best.pt"
                                                torch.save(model.state_dict(), bestRVweights)


4005it [00:03, 1274.05it/s]
4005it [00:00, 6706.81it/s]
 31%|████         | 40/128 [01:21<01:48,  1.24s/it, train0.0 total0.2792 dsc0.7963 ce0.2792 constrain77.3234  (0.1920)]Exception in thread QueueFeederThread:
Traceback (most recent call last):
  File "C:\Users\Windows\Anaconda3\lib\multiprocessing\queues.py", line 232, in _feed
    close()
  File "C:\Users\Windows\Anaconda3\lib\multiprocessing\connection.py", line 177, in close
    self._close()
  File "C:\Users\Windows\Anaconda3\lib\multiprocessing\connection.py", line 277, in _close
    _CloseHandle(self._handle)
OSError: [WinError 6] The handle is invalid

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Windows\Anaconda3\lib\threading.py", line 926, in _bootstrap_inner
    self.run()
  File "C:\Users\Windows\Anaconda3\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Windows\Anaconda3\lib\multiprocessing\que

this is the best so far tensor(0.1158, device='cuda:0', dtype=torch.float64)


100%|████████████████| 25/25 [00:56<00:00,  1.18it/s, val1.0 total0.0843 dsc0.3901 ce0.0843 constrain38.1658  (0.0942)]

this is the best so far tensor(0.0843, device='cuda:0', dtype=torch.float64)


100%|████████████████| 25/25 [00:59<00:00,  1.11it/s, val2.0 total0.0715 dsc0.3169 ce0.0715 constrain33.1695  (0.0714)]

...

100%|████████████████| 25/25 [00:55<00:00,  1.35it/s, val45.0 total0.0000 dsc0.8575 ce0.6454 constrain0.0000  (0.0000)]

this is the best so far tensor(1.0532e-05, device='cuda:0', dtype=torch.float64)


100%|████████████████| 25/25 [00:56<00:00,  1.09it/s, val46.0 total0.0000 dsc0.8577 ce0.6453 constrain0.0000  (0.0000)]

this is the best so far tensor(8.3405e-06, device='cuda:0', dtype=torch.float64)


100%|████████████████| 25/25 [00:56<00:00,  2.25s/it, val49.0 total0.0000 dsc0.8580 ce0.6432 constrain0.0000  (0.0000)]


In [12]:
# constrainedloss testing with smaller loss, same sweep region

#bestRVweights = "C:\\Datasets\RVWork\\RV_deeplab_50_SQLguide_True_AugmentTrue_best.pt"
bestLVweights = "C:\\Users\\Windows\\Dropbox\\Echo Research\\CodeBase\\EchoNetDynamic-Weights\\deeplabv3_resnet50_random.pt"

numWork = 10
numberOfSubsequentFrames = 5
bs = 45
numEpochs = 100

for weightOfConstraint in  [0.8, 0.85, 0.9, 0.95]: #[0, 0.25, 0.1, 0.5, 0.75, 0.9, 1]: #  0.5, 0.1, 0, 0.9, 1 ]: #1, 0,  0.9, #   1000, 100,  10, 1]:
    for CrossEntropyLoss in [1]:
        for SonoData in [False]:
            for doAugmentation in [False]:
                for doPretrain in [False]:


                    train_dataset = dataset.Dataset(SQLtracings, SQLvideos, mean, std, split="train", fuzzyAugmentation=doAugmentation,constrainSubsequent=numberOfSubsequentFrames,useSonoData=False)

                    if SonoData:
                        sono_dataset = dataset.Dataset(SQLtracings, SQLvideos, mean, std, split="train", fuzzyAugmentation=doAugmentation,constrainSubsequent=numberOfSubsequentFrames,useSonoData=True)
                        train_dataloader = torch.utils.data.DataLoader(torch.utils.data.ConcatDataset([train_dataset,sono_dataset]), batch_size = bs, num_workers = numWork, shuffle = True, pin_memory = True, drop_last = True)
                    else:
                        train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size = bs, num_workers = numWork, shuffle = True, pin_memory = True, drop_last = True)

                    val_dataset = dataset.Dataset(SQLtracings, SQLvideos, mean, std, split="val", fuzzyAugmentation=False,constrainSubsequent=numberOfSubsequentFrames,useSonoData=False)
                    val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size = bs, num_workers = numWork, shuffle = True, pin_memory = True, drop_last = True)

                    dataloaders = {'train':train_dataloader, 'val':val_dataloader}

                    device = torch.device("cuda")

                    model = torchvision.models.segmentation.deeplabv3_resnet50(pretrained = False, aux_loss = False)
                    model.classifier[-1] = torch.nn.Conv2d(model.classifier[-1].in_channels, 1, kernel_size = model.classifier[-1].kernel_size)

                    if device.type == "cuda":
                        model = torch.nn.DataParallel(model)
                    model.to(device)

                    if doPretrain:
                        print("load from weights")
                        checkpoint = torch.load(bestLVweights)#LVWeights) #currentRVweights)#
                        model.load_state_dict(checkpoint['state_dict'])

                    #optim = torch.optim.SGD(model.parameters(), lr=1000, momentum=0.9)
                    learning_rate = 1e-2
                    optim = torch.optim.Adam(model.parameters(), lr=learning_rate)
                    #optim = torch.optim.Adagrad(model.parameters(), lr=100)

                    bestLoss = 1000000

                    with open("LossAcrossEpochs_Pretrain"+str(doPretrain)+"_Sono"+str(SonoData)+"_Augment"+str(doAugmentation)+ "_withCrossEntropyOnly"+ str(CrossEntropyLoss) + "_withL2ConstrainedLoss"+ str(weightOfConstraint) + "_"+str(numEpochs)+"_Adam.csv", 'w') as output:
                        for epoch in range(numEpochs):
                            for phase in dataloaders.keys():

                                total = 0
                                n = 0
                                dsctotal = 0
                                cetotal = 0
                                constrainedTotal = 0

                                with torch.set_grad_enabled(phase != 'val'):
                                    with tqdm.tqdm(total = len(dataloaders[phase])) as progressbar:
                                        for (i, (x,y)) in enumerate(dataloaders[phase]):

                                            x = x.to(device)
                                            y = y.to(device)
                                            #print(y.min(), y.max())
                                            #print(x.shape)

                                            #print("input shape", x.shape)

                                            # reshape to be bs x frames indepedent predictions on (3,112,112) images
                                            x = torch.transpose(x, 1, 4)
                                            #print(x.shape)
                                            x = x.reshape(bs * numberOfSubsequentFrames,  112, 112, 3).transpose(1,2)
                                            x = torch.transpose(x, 1, 3).type(torch.float)

                                            #print("new input shape", x.shape)

                                            p = model(x)["out"]

                                            #print("predictions shape", p.shape)

                                            constrainedLoss = variousLossFunctions.constrainSubsequentLoss(p, bs, numberOfSubsequentFrames) / 112 / 112
                                            dscloss = x.shape[0] / numberOfSubsequentFrames - variousLossFunctions.diceLoss(p[0::numberOfSubsequentFrames, 0, :, :], y,)
                                            celoss = torch.nn.functional.binary_cross_entropy_with_logits(p[0::numberOfSubsequentFrames, 0, :, :], y, reduction = "sum") / 112 / 112

                                            loss =  (1-weightOfConstraint) * celoss + weightOfConstraint * constrainedLoss
                                            #CrossEntropyLoss * torch.nn.functional.binary_cross_entropy_with_logits(p[0::numberOfSubsquentFrames, 0, :, :], y, reduction = "sum") / 112 / 112 + variousLossFunctions.constrainSubsequentLoss(p, bs, numberOfSubsequentFrames).item()#x.shape[0] - variousLossFunctions.diceLoss(p[:, 0, :, :], y,) +
                                            
#                                            loss = loss.item()
                                            total += loss#.item()
                                            dsctotal += dscloss#.item()
                                            cetotal += celoss#.item()
                                            constrainedTotal += constrainedLoss

                                            n += x.shape[0] / numberOfSubsequentFrames

                                            #print(loss, total, n, flush = True)

                                            progressbar.set_postfix_str(phase + "{:.1f} total{:.4f} dsc{:.4f} ce{:.4f} constrain{:.4f}  ({:.4f})".format( epoch, total / n , dsctotal / n , cetotal / n , constrainedTotal / n, loss / x.size(0) * numberOfSubsequentFrames )) #/ 112 / 112
                                            progressbar.update()

                                            if(phase != 'val'):
                                                optim.zero_grad()
                                                loss.backward()
                                                optim.step()

                                        output.write(phase + "," + str(epoch) + "," + str(total.item() / n ) + "," + str(dsctotal.item() / n ) + "," + str(cetotal.item() / n ) + ',' + str(constrainedTotal.item() / n) + "\n") #/ 112 / 112

                                        if phase == 'val':
                                            currentRVweights = "C:\\Datasets\RVWork\\RV_deeplab_50_Pretrain"+str(doPretrain)+"_Sono"+str(SonoData)+"_Augment"+str(doAugmentation)+ "_crossEntropyOnly"+ str(CrossEntropyLoss) + "_withL2ConstrainedLoss"+ str(weightOfConstraint)  + "_"+str(numEpochs)+"_Adam_current.pt"
                                            torch.save(model.state_dict(), currentRVweights)

                                            if (total / n ) < bestLoss: #/ 112 / 112)
                                                bestLoss = (total / n ) # / 112 / 112)
                                                print("this is the best so far", str(bestLoss), flush = True)
                                                bestRVweights = "C:\\Datasets\RVWork\\RV_deeplab_50_Pretrain"+str(doPretrain)+"_Sono"+str(SonoData)+"_Augment"+str(doAugmentation)+ "_crossEntropyOnly"+ str(CrossEntropyLoss) +"_withL2ConstrainedLoss"+ str(weightOfConstraint)  + "_"+str(numEpochs)+"_Adam_best.pt"
                                                torch.save(model.state_dict(), bestRVweights)


4005it [00:19, 202.72it/s]
4005it [00:03, 1015.41it/s]
100%|█████████████████| 25/25 [00:56<00:00,  1.23it/s, val0.0 total0.0203 dsc0.4613 ce0.0981 constrain0.0009  (0.0210)]

this is the best so far tensor(0.0203, device='cuda:0', dtype=torch.float64)


100%|█████████████████| 25/25 [00:57<00:00,  1.06it/s, val1.0 total0.0155 dsc0.3498 ce0.0736 constrain0.0009  (0.0167)]

this is the best so far tensor(0.0155, device='cuda:0', dtype=torch.float64)


100%|█████████████████| 25/25 [00:56<00:00,  1.16it/s, val2.0 total0.0141 dsc0.2909 ce0.0655 constrain0.0013  (0.0137)]

...

this is the best so far tensor(0.0032, device='cuda:0', dtype=torch.float64)


100%|████████████████| 25/25 [00:57<00:00,  1.12it/s, val14.0 total0.0032 dsc0.1971 ce0.0485 constrain0.0008  (0.0033)]

this is the best so far tensor(0.0032, device='cuda:0', dtype=torch.float64)


100%|████████████| 128/128 [03:14<00:00,  1.52s/it, train82.0 total0.0023 dsc0.1353 ce0.0331 constrain0.0007  (0.0019)]


100%|████████████████| 25/25 [00:56<00:00,  2.25s/it, val99.0 total0.0048 dsc0.1521 ce0.0519 constrain0.0023  (0.0046)]


In [ ]:
# hyperparameter search without constrained loss

#bestRVweights = "C:\\Datasets\RVWork\\RV_deeplab_50_SQLguide_True_AugmentTrue_best.pt"
bestLVweights = "C:\\Users\\Windows\\Dropbox\\Echo Research\\CodeBase\\EchoNetDynamic-Weights\\deeplabv3_resnet50_random.pt"

for diceLoss in [0,1]:
    for CrossEntropyLoss in [0,1]:
        for SonoData in [ False, True]:
            for doAugmentation in [False, True]:
                for doPretrain in [True]:
                    if(diceLoss + CrossEntropyLoss > 0):
                        print(diceLoss, CrossEntropyLoss, SonoData, doAugmentation, doPretrain)
                        
                        train_dataset = dataset.Dataset(SQLtracings, SQLvideos, mean, std, split="train", fuzzyAugmentation=doAugmentation,constrainSubsequent=0,useSonoData=False)

                        if SonoData:
                            sono_dataset = dataset.Dataset(SQLtracings, SQLvideos, mean, std, split="train", fuzzyAugmentation=doAugmentation,constrainSubsequent=0,useSonoData=True)
                            train_dataloader = torch.utils.data.DataLoader(torch.utils.data.ConcatDataset([train_dataset,sono_dataset]), batch_size = 100, num_workers = 5, shuffle = True, pin_memory = True, drop_last = True)
                        else:
                            train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size = 100, num_workers = 5, shuffle = True, pin_memory = True, drop_last = True)

                        val_dataset = dataset.Dataset(SQLtracings, SQLvideos, mean, std, split="val", fuzzyAugmentation=False,constrainSubsequent=0,useSonoData=False)
                        val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size = 100, num_workers = 5, shuffle = True, pin_memory = True, drop_last = True)

                        dataloaders = {'train':train_dataloader, 'val':val_dataloader}

                        device = torch.device("cuda")

                        model = torchvision.models.segmentation.deeplabv3_resnet50(pretrained = False, aux_loss = False)
                        model.classifier[-1] = torch.nn.Conv2d(model.classifier[-1].in_channels, 1, kernel_size = model.classifier[-1].kernel_size)

                        if device.type == "cuda":
                            model = torch.nn.DataParallel(model)
                        model.to(device)

                        if doPretrain:
                            print("load from weights")
                            checkpoint = torch.load(bestLVweights)#LVWeights) #currentRVweights)#
                            model.load_state_dict(checkpoint['state_dict'])

                        learning_rate = 1e-4
                        optim = torch.optim.Adam(model.parameters(), lr=learning_rate)

                        # optim = torch.optim.Adagrad(model.parameters(), lr=100)

                        bestLoss = 1000000

                        with open("LossAcrossEpochs_Pretrain"+str(doPretrain)+"_Sono"+str(SonoData)+"_Augment"+str(doAugmentation)+ "_diceLoss"+ str(diceLoss) + "_CrossEntropyLoss"+ str(CrossEntropyLoss) + "_Adam.csv", 'w') as output:
                            for epoch in range(50):
                                for phase in dataloaders.keys():

                                    total = 0
                                    n = 0
                                    dsctotal = 0
                                    cetotal = 0

                                    with torch.set_grad_enabled(phase != 'val'):
                                        with tqdm.tqdm(total = len(dataloaders[phase])) as progressbar:
                                            for (i, (x,y)) in enumerate(dataloaders[phase]):

                                                x = x.to(device)
                                                y = y.to(device)
                                                #print(y.min(), y.max())
                                                #print(x.shape)
                                                p = model(x)["out"]
                                                loss = diceLoss * (x.shape[0] - variousLossFunctions.diceLoss(p[:, 0, :, :], y,)) + CrossEntropyLoss * torch.nn.functional.binary_cross_entropy_with_logits(p[:, 0, :, :], y, reduction = "sum") / 112 / 112 #+ variousLossFunctions.diceLoss(p[:, 0, :, :], y,)
                                                dscloss = x.shape[0] - variousLossFunctions.diceLoss(p[:, 0, :, :], y,)
                                                celoss = torch.nn.functional.binary_cross_entropy_with_logits(p[:, 0, :, :], y, reduction = "sum") / 112 / 112

                                                total += loss.item()
                                                dsctotal += dscloss#.item()
                                                cetotal += celoss#.item()

                                                n += x.shape[0]

                                                #print(loss, total, n, flush = True)

                                                progressbar.set_postfix_str(phase + "{:.1f} {:.4f} {:.4f} {:.4f}  ({:.4f})".format( epoch, total / n , dsctotal / n , cetotal / n , loss.item() / x.size(0) )) #/ 112 / 112
                                                progressbar.update()

                                                if(phase != 'val'):
                                                    optim.zero_grad()
                                                    loss.backward()
                                                    optim.step()

                                            output.write(phase + "," + str(epoch) + "," + str(total / n ) + "," + str(dsctotal.item() / n ) + "," + str(cetotal.item() / n ) + "\n") #/ 112 / 112

                                            if phase == 'val':
                                                currentRVweights = "C:\\Datasets\RVWork\\RV_deeplab_50_Pretrain"+str(doPretrain)+"_Sono"+str(SonoData)+"_Augment"+str(doAugmentation)+ "_diceLoss"+ str(diceLoss) + "_CrossEntropyLoss"+ str(CrossEntropyLoss) + "_Adam_current.pt"
                                                torch.save(model.state_dict(), currentRVweights)

                                                if (total / n ) < bestLoss: #/ 112 / 112)
                                                    bestLoss = (total / n ) # / 112 / 112)
                                                    print("this is the best so far", str(bestLoss), flush = True)
                                                    bestRVweights = "C:\\Datasets\RVWork\\RV_deeplab_50_Pretrain"+str(doPretrain)+"_Sono"+str(SonoData)+"_Augment"+str(doAugmentation)+ "_diceLoss"+ str(diceLoss) + "_CrossEntropyLoss"+ str(CrossEntropyLoss) +"_Adam_best.pt"
                                                    torch.save(model.state_dict(), bestRVweights)


In [13]:
def collate_fn(x):
    x, f = zip(*x)
    i = list(map(lambda t: t.shape[1], x))
    x = torch.as_tensor(np.swapaxes(np.concatenate(x, 1), 0, 1))
    return x, f, i


In [14]:
# Generate videos from both test set of RV focused Views

device = torch.device("cuda")
testFolder = "C:\\Users\\Windows\\Dropbox\\Echo Research\\CodeBase\\DynamicRV\\Constrained\\SmallTestSet"


dataloader = torch.utils.data.DataLoader(echonet.datasets.Echo(split="external_test", external_test_location = testFolder, target_type=["Filename"], length=None, period=1, mean=mean, std=std),
                                        batch_size=100, num_workers=0, shuffle=False, pin_memory=(device.type == "cuda"), collate_fn=collate_fn)


# test_dataset = dataset.Dataset(SQLtracings, SQLvideos, mean, std, split="test", fuzzyAugmentation=False,constrainSubsequent=0,useSonoData=False)
# dataloader = torch.utils.data.DataLoader(test_dataset, batch_size = 100, num_workers = 0, shuffle = False, pin_memory=(device.type == "cuda"), collate_fn=collate_fn)
          
stem = "C:\\Datasets\\RVWork\\RV_deeplab_50_PretrainFalse_SonoFalse_AugmentFalse_crossEntropyOnly1_with"
# foldername = ["FullFeatures_DiceLoss_RVViews","FullFeatures_CELoss_RVViews","FullFeatures_BothLoss_RVViews"]
loss = ["L2", "L1"]
weight = ["1", "0.9", "0.75", "0.5", "0.25", "0.1", "0"]
folder = "Constrained\\ExamplesWithBiggerLoss"
bestRVweights = ["ConstrainedLoss1_50_Adam_best.pt",
                "ConstrainedLoss0.9_50_Adam_best.pt",
                 "ConstrainedLoss0.75_50_Adam_best.pt",
                 "ConstrainedLoss0.5_50_Adam_best.pt",
                 "ConstrainedLoss0.25_50_Adam_best.pt",
                 "ConstrainedLoss0.1_50_Adam_best.pt",
                 "ConstrainedLoss0_50_Adam_best.pt"]


model = torchvision.models.segmentation.deeplabv3_resnet50(pretrained = False, aux_loss = False)
model.classifier[-1] = torch.nn.Conv2d(model.classifier[-1].in_channels, 1, kernel_size = model.classifier[-1].kernel_size)

if device.type == "cuda":
     model = torch.nn.DataParallel(model)
model.to(device)

for thisLoss in loss:
    for count, thisweight in enumerate(bestRVweights):
        print(count, thisLoss, thisweight)
        checkpoint = torch.load(stem + thisLoss + thisweight)
        model.load_state_dict(checkpoint)


        #all_dataset = dataset.Dataset(SQLtracings, SQLvideos, mean, std, split="all", fuzzyAugmentation=False,constrainSubsequent=0)
        #dataloader = torch.utils.data.DataLoader(all_dataset, batch_size = 200, num_workers = 10, shuffle = True, pin_memory = True, drop_last = True)

        #bestRVweights = "C:\\Datasets\RVWork\\RV_deeplab_50_Sono_True_Pretrain_False_Augment_False_best.pt"
        #
        #checkpoint = torch.load(bestRVweights)
        #model.load_state_dict(checkpoint['state_dict'])

        #os.makedirs(folder, exist_ok=True)

        block = 256
        with torch.no_grad():
            for (x, f, i) in tqdm.tqdm(dataloader):
                x = x.to(device)
                y = np.concatenate([model(x[i:(i + block), :, :, :])["out"].detach().cpu().numpy() for i in range(0, x.shape[0], block)]).astype(np.float16)

                start = 0
                x = (x.cpu().numpy() * std.reshape((1, 3, 1, 1))) + mean.reshape((1, 3, 1, 1))

                for (filename, offset) in zip(f, i):
                    img = x[start:(start + offset), ...].transpose((1, 0, 2, 3))
                    img = np.concatenate((img, img), 3)
                    logit = y[start:(start + offset), 0, :, :].copy()
                    img[1, :, :, 112:] = np.maximum(255. * (logit > 0), img[0, :, :, 112:])

                    #img = y[start:(start + offset)]
                    #np.save(os.path.join(output, "labels", os.path.splitext(filename)[0]), y[start:(start + offset), 0, :, :])
                    echonet.utils.savevideo(os.path.join(folder, os.path.splitext(filename)[0] + "_" + thisLoss + "_"+ weight[count] + ".avi"), img.astype(np.uint8), 50)

                    start += offset

0 L2 ConstrainedLoss1_50_Adam_best.pt


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.96s/it]


1 L2 ConstrainedLoss0.9_50_Adam_best.pt


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.73s/it]


2 L2 ConstrainedLoss0.75_50_Adam_best.pt


...

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.77s/it]


6 L2 ConstrainedLoss0_50_Adam_best.pt


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.75s/it]


0 L1 ConstrainedLoss1_50_Adam_best.pt


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Datasets\\RVWork\\RV_deeplab_50_PretrainFalse_SonoFalse_AugmentFalse_crossEntropyOnly1_withL1ConstrainedLoss1_50_Adam_best.pt'